In [ ]:
import ee
import geemap

In [16]:
# authenticate and initialize Earth Engine

ee.Authenticate()
ee.Initialize(project="gsapp-map")

dataset = ee.ImageCollection("GOOGLE/SATELLITE_EMBEDDING/V1/ANNUAL")

center = [34.80251148191422, 31.6008201894651]
point = ee.Geometry.Point(center[0], center[1])
# image2024 = dataset.filterDate("2024-01-01", "2025-01-01").filterBounds(region).mosaic()

region = point.buffer(500000)

image2017 = dataset.filterDate("2017-01-01", "2018-01-01").filterBounds(region).mosaic()
image2022 = dataset.filterDate("2022-01-01", "2023-01-01").filterBounds(region).mosaic()
image2024 = dataset.filterDate("2024-01-01", "2025-01-01").filterBounds(region).mosaic()

# Visualize three axes of the embedding space as an RGB.
visParams = {min: -0.3, max: 0.3, "bands": ["A44", "A18", "A50"]}

Map = geemap.Map(center=center, zoom=4)
Map.add_basemap("TopPlusOpen.Grey")

Map.addLayer(image2017, visParams, "2017 embeddings")
Map.addLayer(image2024, visParams, "2024 embeddings")

Map.centerObject(point, zoom=6)
Map.setOptions("SATELLITE")
Map

Map(center=[31.6008201894651, 34.80251148191422], controls=(WidgetControl(options=['position', 'transparent_bg…

In [19]:
# This section is unrelated to the rest of the notebook
# Visualize three axes of the embedding space as an RGB (for export or display in Colab)

# Sample points for training
n_samples = 1000
training2024 = image2024.sample(
    region=region, scale=10, numPixels=n_samples, seed=100, geometries=True
)
training2017 = image2017.sample(
    region=region, scale=10, numPixels=n_samples, seed=100, geometries=True
)


# Function to train a model for desired number of clusters
def get_clusters(training, n_clusters):
    clusterer = ee.Clusterer.wekaKMeans(n_clusters).train(training)
    clustered = image2024.cluster(clusterer)
    return clustered


cluster10_2024 = get_clusters(training2024, 10)
cluster20_2024 = get_clusters(training2024, 20)
clusters40_2024 = get_clusters(training2024, 40)

cluster10_2017 = get_clusters(training2017, 10)
cluster20_2017 = get_clusters(training2017, 20)
clusters40_2017 = get_clusters(training2017, 40)

Map = geemap.Map(center=center, zoom=6)
Map.add_basemap("TopPlusOpen.Grey")

# Add clusters
Map.addLayer(cluster10_2024.clip(region).randomVisualizer(), {}, "2024: 10 Clusters")
Map.addLayer(cluster20_2024.clip(region).randomVisualizer(), {}, "2024: 20 Clusters")
Map.addLayer(clusters40_2024.clip(region).randomVisualizer(), {}, "2024: 40 Clusters")

Map.addLayer(cluster10_2017.clip(region).randomVisualizer(), {}, "2017: 10 Clusters")
Map.addLayer(cluster20_2017.clip(region).randomVisualizer(), {}, "2017: 20 Clusters")
Map.addLayer(clusters40_2017.clip(region).randomVisualizer(), {}, "2017: 40 Clusters")

Map

Map(center=[34.80251148191422, 31.6008201894651], controls=(WidgetControl(options=['position', 'transparent_bg…

In [17]:
# Compute per-pixel Euclidean distance between embedding vectors
bands = list(
    set(image2022.bandNames().getInfo()) & set(image2024.bandNames().getInfo())
)
diff = image2024.select(bands).subtract(image2022.select(bands))
sq = diff.pow(2)
sum_sq = sq.reduce(ee.Reducer.sum())
euclidean_dist = sum_sq.sqrt().rename("euclidean_dist")

Map = geemap.Map(center=center, zoom=6)
Map.add_basemap("TopPlusOpen.Grey")

# Visualize the Euclidean distance (difference magnitude)
Map.addLayer(
    euclidean_dist,
    {"min": 0, "max": 1, "palette": ["black", "white"]},
    "Embedding Difference (Euclidean)",
)

Map.centerObject(point, zoom=9)
Map

Map(center=[31.6008201894651, 34.80251148191422], controls=(WidgetControl(options=['position', 'transparent_bg…